In [10]:
import numpy as np
import EvalScript.evalResult as evalResult
import heapq
import math
import random
import matplotlib.pyplot as plt

# Ignore warning for np.log(0)
np.seterr(under="warn", divide="ignore")

BASE = './'
START = 'START'
STOP = 'STOP'

In [11]:
class UnlabelledData:
    def __init__(self, path=None):
        self.sentences = []
        if path == None:
            return
        with open(path, 'r', encoding='utf8') as f:
            current_sentence = []
            for line in f:
                line = line.strip()
                if line == '':
                    self.sentences.append(current_sentence)
                    current_sentence = []
                else:
                    current_sentence.append(line)
            if len(current_sentence):
                self.sentences.append(current_sentence)

In [12]:
class LabelledData:
    def __init__(self, path = None):
        self.sentences = []
        if path == None:
            return
        with open(path, 'r', encoding='utf8') as f:
            current_sentence = []
            for line in f:
                line = line.strip()
                if line == '':
                    self.sentences.append(current_sentence)
                    current_sentence = []
                else:
                    current_sentence.append(tuple(line.rsplit(maxsplit=1)))
            if len(current_sentence):
                self.sentences.append(current_sentence)
    
    def to_unlabelled(self):
        unlabelled = UnlabelledData()
        unlabelled.sentences = list(map(lambda s: list(map(lambda x: x[0], s)), self.sentences))
        return unlabelled
    
    def write_to_file(self, path):
        with open(path, 'w', encoding='utf8') as f:
            for sentence in self.sentences:
                for data in sentence:
                    print(*data, file=f)
                print(file=f)

# Part 1

In [13]:
class HMModel1:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive']
    
    def __init__(self, k):
        self.emit_counts = {}
        self.label_counts = dict.fromkeys(self.labels, 0)
        self.k = k
    
    def get_emission_prob(self, x, y):
        return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))
    
    def learn(self, data: LabelledData):
        for sentence in data.sentences:
            for x, y in sentence:
                for label in self.labels:
                    self.emit_counts.setdefault((label, x), 0)
                self.emit_counts[y, x] += 1
                self.label_counts[y] += 1
    
    def label(self, data: UnlabelledData):
        labelled = LabelledData()
        for unlabelled_sentence in data.sentences:
            sentence = []
            for token in unlabelled_sentence:
                y_max = None; y_max_prob = float('-inf')
                for y in self.labels:
                    y_prob = self.get_emission_prob(token, y)
                    if y_prob > y_max_prob:
                        y_max_prob = y_prob
                        y_max = y

                sentence.append((token, y_max))
            labelled.sentences.append(sentence)
        return labelled
            

In [14]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')
    model = HMModel1(k=1)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/dev.p1.out')
    
    print(f'{f" {dataset} ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p1.out'))
    print('='*30)

============= RU =============
#Entity in gold data: 389
#Entity in prediction: 1816

#Correct Entity : 266
Entity  precision: 0.1465
Entity  recall: 0.6838
Entity  F: 0.2413

#Correct Sentiment : 129
Sentiment  precision: 0.0710
Sentiment  recall: 0.3316
Sentiment  F: 0.1170


TypeError: EvalScript.evalResult.printResult() argument after * must be an iterable, not NoneType

# Part 2

In [15]:
class HMModel2:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive', START, STOP]
    
    def __init__(self, k):
        self.emit_counts = {}
        self.label_counts = dict.fromkeys(self.labels, 0)
        self.transition_counts = {}
        self.k = k
        self.label_to_index = dict(map(lambda x: (x[1], x[0]), enumerate(self.labels)))
    
    def get_emission_prob(self, x, y):
        return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))
    
    def get_transition_prob(self, y1, y2):
        return np.log(float(self.transition_counts.get((y1, y2), 0))) - np.log(float((self.label_counts.get(y1, 1))))
    
    def learn(self, data: LabelledData):
        for sentence in data.sentences:
            prev_y = START
            for x, y in sentence:
                for label in self.labels:
                    self.emit_counts.setdefault((label, x), 0)

                
                self.emit_counts[y, x] += 1
                self.label_counts[y] += 1
                self.transition_counts[prev_y, y] = self.transition_counts.get((prev_y, y), 0) + 1
                prev_y = y
            self.transition_counts[prev_y, STOP] = self.transition_counts.get((prev_y, STOP), 0) + 1

            self.label_counts[START] += 1
            self.label_counts[STOP] += 1
    
    def viterbi(self, sentence, transition_prob):
        pi = np.full((len(sentence) + 1, len(self.labels)), -np.inf, dtype=np.double)
        pi[0, self.label_to_index[START]] = 0.0 # = log(1)

        for k in range(1, len(sentence) + 1):
            token = sentence[k-1]
            for vi, v in enumerate(self.labels):
                possible_next = pi[k-1, :] + transition_prob[:, vi].T + self.get_emission_prob(token, v)
                pi[k, vi] = np.max(possible_next)

        return pi

    def label(self, data):
        labelled = LabelledData()
        transition_prob = np.ndarray(shape=(len(self.labels), len(self.labels)), dtype=np.double)
        
        for ui, u in enumerate(self.labels):
            for vi, v in enumerate(self.labels):
                transition_prob[ui, vi] = self.get_transition_prob(u, v)
        
        for unlabelled_sentence in data.sentences:
            pi = self.viterbi(unlabelled_sentence, transition_prob)
            sentence = list(unlabelled_sentence)
            next_yi = self.label_to_index[STOP]
            for i in reversed(range(len(unlabelled_sentence))):
                next_yi = np.argmax(pi[i+1,:] + transition_prob[:, next_yi].T)
                sentence[i] = (sentence[i], self.labels[next_yi])
            labelled.sentences.append(sentence)
        return labelled

In [16]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')
    model = HMModel2(k=1)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/dev.p2.out')
    
    print(f'{f" {dataset} ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p2.out'))
    print('='*30)

============= RU =============
#Entity in gold data: 389
#Entity in prediction: 488

#Correct Entity : 189
Entity  precision: 0.3873
Entity  recall: 0.4859
Entity  F: 0.4310

#Correct Sentiment : 129
Sentiment  precision: 0.2643
Sentiment  recall: 0.3316
Sentiment  F: 0.2942


TypeError: EvalScript.evalResult.printResult() argument after * must be an iterable, not NoneType

# Part 3

Note here that $k$ has been renamed to $t$ to remove the conflict between the $k$ variable in Viterbi's algorithm.

In [17]:
class HMModel3:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive', START, STOP]
    
    def __init__(self, k):
        self.emit_counts = {}
        self.label_counts = dict.fromkeys(self.labels, 0)
        self.transition_counts = {}
        self.k = k
        self.label_to_index = dict(map(lambda x: (x[1], x[0]), enumerate(self.labels)))
    
    def get_emission_prob(self, x, y):
        return np.log(float(self.emit_counts.get((y, x), self.k))) - np.log(float((self.label_counts[y] + self.k)))
    
    def get_transition_prob(self, y1, y2):
        return np.log(float(self.transition_counts.get((y1, y2), 0))) - np.log(float((self.label_counts.get(y1, 1))))
    
    def learn(self, data: LabelledData):
        for sentence in data.sentences:
            prev_y = START
            for x, y in sentence:
                for label in self.labels:
                    self.emit_counts.setdefault((label, x), 0)

                self.emit_counts[y, x] += 1
                self.label_counts[y] += 1
                self.transition_counts[prev_y, y] = self.transition_counts.get((prev_y, y), 0) + 1
                prev_y = y
            self.transition_counts[prev_y, STOP] = self.transition_counts.get((prev_y, STOP), 0) + 1

            self.label_counts[START] += 1
            self.label_counts[STOP] += 1
    
    def kbest(self, pi, prev, transition_prob, t, k, vi, token=None):
        heap = []
        for i in range(t):
            emission_prob = self.get_emission_prob(token, self.labels[vi]) if token != None else 0.0
            possible_next = pi[k-1, :, i] + transition_prob[:, vi].T + emission_prob

            for j, p in enumerate(possible_next):
                if not any(math.isclose(x[0], p) for x in heap):
                    if len(heap) == t:
                        heapq.heappushpop(heap, (p, i, j))
                    else:
                        heapq.heappush(heap, (p, i, j))

        for besti, (p, i, j) in enumerate(heapq.nlargest(t, heap)):
            pi[k, vi, besti] = p
            prev[k-1, vi, besti, 0] = i
            prev[k-1, vi, besti, 1] = j

    
    def viterbi(self, sentence, transition_prob, t):
        pi = np.full((len(sentence) + 2, len(self.labels), t), -np.inf, dtype=np.double)
        pi[0, self.label_to_index[START], 0] = 0.0 # = log(1)
        prev = np.full((len(sentence) + 1, len(self.labels), t, 2), -1, dtype=np.int64)

        for k in range(1, len(sentence) + 1):
            for vi, v in enumerate(self.labels):
                self.kbest(pi, prev, transition_prob, t, k, vi, sentence[k-1])
        
        self.kbest(pi, prev, transition_prob, t, len(sentence) + 1, self.label_to_index[STOP])

        return prev

    def label(self, data, t):
        labelled = LabelledData()
        transition_prob = np.ndarray(shape=(len(self.labels), len(self.labels)), dtype=np.double)
        
        for ui, u in enumerate(self.labels):
            for vi, v in enumerate(self.labels):
                transition_prob[ui, vi] = self.get_transition_prob(u, v)
        
        for unlabelled_sentence in data.sentences:
            prev = self.viterbi(unlabelled_sentence, transition_prob, t)

            sentence = list(unlabelled_sentence)
            next_yi = self.label_to_index[STOP]
            j = t-1
            
            for i in reversed(range(len(unlabelled_sentence))):
                p = (j, self.labels[next_yi])
                j, next_yi = prev[i+1, next_yi, j, :]
                sentence[i] = (sentence[i], self.labels[next_yi])
            labelled.sentences.append(sentence)
        return labelled

In [18]:
%%time

# Verify the algorithm gives the same result as non-modified viterbi for k=1
for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')

    model = HMModel3(k=1)
    model.learn(train)

    predicted2 = model.label(dev_in, 1)
    predicted2.write_to_file(BASE + dataset + '/dev.p3.1st.out')    
    print(f'{f" {dataset} k=1 ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p3.1st.out'))
    print('='*30)

=========== RU k=1 ===========
#Entity in gold data: 389
#Entity in prediction: 488

#Correct Entity : 189
Entity  precision: 0.3873
Entity  recall: 0.4859
Entity  F: 0.4310

#Correct Sentiment : 129
Sentiment  precision: 0.2643
Sentiment  recall: 0.3316
Sentiment  F: 0.2942


TypeError: EvalScript.evalResult.printResult() argument after * must be an iterable, not NoneType

In [19]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')

    model = HMModel3(k=1)
    model.learn(train)

    predicted2 = model.label(dev_in, 2)
    predicted2.write_to_file(BASE + dataset + '/dev.p3.2nd.out')    
    print(f'{f" {dataset} k=2 ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p3.2nd.out'))
    print('='*30)
    
    predicted8 = model.label(dev_in, 8)
    predicted8.write_to_file(BASE + dataset + '/dev.p3.8th.out')
    print(f'{f" {dataset} k=8 ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p3.8th.out'))
    print('='*30)

=========== RU k=2 ===========
#Entity in gold data: 389
#Entity in prediction: 749

#Correct Entity : 202
Entity  precision: 0.2697
Entity  recall: 0.5193
Entity  F: 0.3550

#Correct Sentiment : 126
Sentiment  precision: 0.1682
Sentiment  recall: 0.3239
Sentiment  F: 0.2214


TypeError: EvalScript.evalResult.printResult() argument after * must be an iterable, not NoneType

# Part 4

In [20]:
class HMModel4:
    labels = ['O', 'B-negative', 'I-negative', 'B-neutral', 'I-neutral', 'B-positive', 'I-positive', START, STOP]
    
    def __init__(self, k):
        self.emit_counts = {}
        self.label_counts = dict.fromkeys(self.labels, 0)
        self.total_label_count = 0
        self.transition_counts = {}
        self.k = k
        self.label_to_index = dict(map(lambda x: (x[1], x[0]), enumerate(self.labels)))
    
    def get_emission_prob(self, x, y):
        # Ignore token case
        x = x.lower()
        if (y, x) in self.emit_counts:
            return np.log(float(self.emit_counts[y, x])) - np.log(float((self.label_counts[y] + self.k)))
        # Use k/(total_count - count(y) + k) as emission probability instead
        return np.log(float(self.k)) - np.log(float(self.total_label_count - self.label_counts[y] + self.k))
    
    def get_transition_prob(self, y1, y2):
        return np.log(float(self.transition_counts.get((y1, y2), 0))) - np.log(float((self.label_counts.get(y1, 1))))
    
    def learn(self, data: LabelledData):
        for sentence in data.sentences:
            prev_y = START
            for x, y in sentence:
                # Ignore token case
                x = x.lower()
                for label in self.labels:
                    self.emit_counts.setdefault((label, x), 0)

                
                self.emit_counts[y, x] += 1
                self.label_counts[y] += 1
                self.transition_counts[prev_y, y] = self.transition_counts.get((prev_y, y), 0) + 1
                prev_y = y
            self.transition_counts[prev_y, STOP] = self.transition_counts.get((prev_y, STOP), 0) + 1

            self.label_counts[START] += 1
            self.label_counts[STOP] += 1
        self.total_label_count = sum(self.label_counts.values())
    
    def viterbi(self, sentence, transition_prob):
        pi = np.full((len(sentence) + 1, len(self.labels)), -np.inf, dtype=np.double)
        pi[0, self.label_to_index[START]] = 0.0 # = log(1)

        for k in range(1, len(sentence) + 1):
            token = sentence[k-1]
            for vi, v in enumerate(self.labels):
                possible_next = pi[k-1, :] + transition_prob[:, vi].T + self.get_emission_prob(token, v)
                pi[k, vi] = np.max(possible_next)

        return pi

    def label(self, data):
        labelled = LabelledData()
        transition_prob = np.ndarray(shape=(len(self.labels), len(self.labels)), dtype=np.double)
        
        for ui, u in enumerate(self.labels):
            for vi, v in enumerate(self.labels):
                transition_prob[ui, vi] = self.get_transition_prob(u, v)
        
        for unlabelled_sentence in data.sentences:
            pi = self.viterbi(unlabelled_sentence, transition_prob)
            sentence = list(unlabelled_sentence)
            next_yi = self.label_to_index[STOP]
            for i in reversed(range(len(unlabelled_sentence))):
                next_yi = np.argmax(pi[i+1,:] + transition_prob[:, next_yi].T)
                sentence[i] = (sentence[i], self.labels[next_yi])
            labelled.sentences.append(sentence)
        return labelled

In [21]:
%%time

for (dataset, k) in [('RU', 1), ('ES', 1)]:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')
    model = HMModel4(k)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/dev.p4.out')
    
    print(f'{f" {dataset} ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p4.out'))
    print('='*30)

============= RU =============
#Entity in gold data: 389
#Entity in prediction: 305

#Correct Entity : 198
Entity  precision: 0.6492
Entity  recall: 0.5090
Entity  F: 0.5706

#Correct Sentiment : 135
Sentiment  precision: 0.4426
Sentiment  recall: 0.3470
Sentiment  F: 0.3890


TypeError: EvalScript.evalResult.printResult() argument after * must be an iterable, not NoneType

In [22]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')
    model = HMModel4(k)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/dev.p4.out')
    
    print(f'{f" {dataset} ":=^30}')
    evalResult.printResult(*evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p4.out'))
    print('='*30)

============= RU =============
#Entity in gold data: 389
#Entity in prediction: 305

#Correct Entity : 198
Entity  precision: 0.6492
Entity  recall: 0.5090
Entity  F: 0.5706

#Correct Sentiment : 135
Sentiment  precision: 0.4426
Sentiment  recall: 0.3470
Sentiment  F: 0.3890


TypeError: EvalScript.evalResult.printResult() argument after * must be an iterable, not NoneType

In [23]:
x_range = range(1, 1001, 5)

for dataset in ['RU', 'ES']:
    fe = []
    mxe = None
    fs = []
    mxs = None
    x = list(x_range)
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/dev.in')
    for k in x_range:
        if mxe != None:
            print(f'Trying k = {k}, best so far = {x[mxe]} (entity), {x[mxs]} (sentiment)', end='\r')
        else:
            print(f'Trying k = {k}', end='\r')
        model = HMModel4(k)
        model.learn(train)
        predicted = model.label(dev_in)
        predicted.write_to_file(BASE + dataset + '/dev.p2-temp.out')

        to, tp, ce, cs = evalResult.evaluate(BASE + dataset + '/dev.out', BASE + dataset + '/dev.p2-temp.out')
        fe.append(evalResult.calculateFscore(ce/tp, ce/to))
        fs.append(evalResult.calculateFscore(cs/tp, cs/to))
        if mxe == None or fe[-1] > fe[mxe]:
            mxe = len(fe)-1
        if mxs == None or fs[-1] > fs[mxs]:
            mxs = len(fs)-1
    print(dataset, f'done, best = {x[mxe]} (entity), {x[mxs]} (sentiment)', ' '*40)
        
    xemax = x[mxe]
    yemax = fe[mxe]
    xsmax = x[mxs]
    ysmax = fs[mxs]
    
    fig, ax = plt.subplots()
    ax.set_title(dataset)
    ax.plot(x, fe, label="Entity F-score")
    ax.plot(x, fs, label="Sentiment F-score")
    ax.annotate('Entity max ' + str(xemax), xy=(xemax, yemax+0.05))
    ax.annotate('Sentiment max ' + str(xsmax), xy=(xsmax, ysmax+0.05))
    ax.set_ybound(0.0, 1.0)
    ax.legend()

#Entity in gold data: 389
#Entity in prediction: 305

#Correct Entity : 198
Entity  precision: 0.6492
Entity  recall: 0.5090
Entity  F: 0.5706

#Correct Sentiment : 135
Sentiment  precision: 0.4426
Sentiment  recall: 0.3470
Sentiment  F: 0.3890


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
%%time

for dataset in ['RU', 'ES']:
    train = LabelledData(BASE + dataset + '/train')
    dev_in = UnlabelledData(BASE + dataset + '/test.in')
    model = HMModel4(k)
    model.learn(train)
    predicted = model.label(dev_in)
    predicted.write_to_file(BASE + dataset + '/test.p4.out')
    print('Finished predicting', dataset)